In [2]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Load data
import load_data as data

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
# n different points we want to cluster
# We have 59 points in all 3 categories * 2 people * 10 repetitions (maia_fall only had 9)
train_trials = 59
test_trials = 16
# Tensor length of 80 since that's how many frames we decided on--easily modifiable
NUM_FRAMES = 80
NUM_FEATURES = 36

# Starting off with just one point so that it's easier to follow along with tutorials
# Will switch to more than one point later
# joint_index specifies which of the 36 joints we want, 0 is nose
# Find corresponding values at https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/output.md
joint_index = 0 

# 3D
# Xtr, Ytr = data.train_get_three_dimension(train_trials)
# Xte, Yte = data.test_get_three_dimension(test_trials)



In [7]:
# Start training
accuracies = np.zeros(NUM_FEATURES)
for i in range(NUM_FEATURES):
    # 2D
    Xtr, Ytr = data.train_get_one_keypoint(train_trials, i)
    Xte, Yte = data.test_get_one_keypoint(test_trials, i)


    # tf Graph Input

    # 3D
    # xtr = tf.placeholder("float", [train_trials, NUM_FRAMES, NUM_FEATURES])
    # xte = tf.placeholder("float", [NUM_FRAMES, NUM_FEATURES])

    # 2D
    xtr = tf.placeholder("float", [train_trials, NUM_FRAMES])
    xte = tf.placeholder("float", [NUM_FRAMES])

    # Nearest Neighbor calculation using L1 Distance
    # Calculate L1 Distance
#     distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)
    
    # Euclidean
    distance = tf.sqrt( tf.reduce_sum(tf.square(tf.subtract(xtr, xte)),
                                reduction_indices=1))
    
    # Cosine
#     distance = tf.reduce_sum(tf.multiply(tf.nn.l2_normalize(xtr ,0), tf.nn.l2_normalize(xte, 0)), reduction_indices=1)
    
    # Prediction: Get min distance index (Nearest neighbor)
    pred = tf.argmin(distance, 0)

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

    ###### Training code

    with tf.Session() as sess:
        accuracy = 0.
        # Run the initializer
        sess.run(init)

        # loop over test data
        for j in range(len(Xte)):
            # Get nearest neighbor
            # 3D
            # nn_index = sess.run(pred, feed_dict={xtr: Xtr, xte: Xte[i, :, :]})

            #2D
            nn_index = sess.run(pred, feed_dict={xtr: Xtr, xte: Xte[j, :]})
            
            # Get nearest neighbor class label and compare it to its true label
#             print("Test", i, "Prediction:", np.argmax(Ytr[nn_index]) + 1, \
#                 "True Class:", np.argmax(Yte[i]) + 1)
            # Calculate accuracy
            if np.argmax(Ytr[nn_index]) == np.argmax(Yte[j]):
                accuracy += 1./len(Xte)
        print("Feature", i, "Accuracy:", accuracy)
        accuracies[i] = accuracy
#         print(distance.eval())

# print(accuracies)   
# plt.plot(accuracies)

ValueError: Shapes (80,) and (59, 80) are incompatible